In [9]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("IRSE_FIRE_2022_Track_Training_Data_preprocessed.csv")

In [3]:
data.tail(100)

,Comments,Surrounding Code Context,Class
7947,"/*only count in first row, since\nall rows sho...",-9. size_type count = 0;\n-8. for (size_ty...,Useful
7948,"/*delete old content, set block\nsizes anew*/",-10. MemoryConsumption::memory_consu...,Useful
7949,/*and finally enquire their new\nsizes*/,collect_sizes();\n}\n\n\n\nBlockDynamicSparsit...,Useful
7950,/*This file compiles a part of the instantiati...,-10. // This file is part of the deal.II libra...,Useful
7951,"/*First, initialize the\ngeneralized support p...",-10. ...,Useful
...,...,...,...
8042,/*we need an additional DG0-node for a dim-dim...,-10. riaf.back() = true;\n-9. return riaf;...,Useful
8043,/*vertex/line/face domination\n(if fe_other is...,-1. return FiniteElementDomination::no_r...,Useful
8044,/*there are no requirements between continuous...,-2. if (codim > 0)\n-1. if (dynamic_cast...,Useful
8045,/*vertex/line/face domination\n(if fe_other is...,-1. return FiniteElementDomination::no_r...,Useful


In [4]:
print(data['Surrounding Code Context'][0])

-10.   int res = 0;
-9.   CURL *curl = NULL;
-8.   FILE *hd_src = NULL;
-7.   int hd;
-6.   struct_stat file_info;
-5.   CURLM *m = NULL;
-4.   int running;
-3.   start_test_timing();
-2.   if(!libtest_arg2) {
-1. #ifdef LIB529

/*test 529*/

1.     fprintf(stderr, "Usage: lib529 [url] [uploadfile]\n");
2. #else


In [61]:
import sys
import re
import glob
import os
import pickle
import emoji

def Diff(a,b):
    return list(set(a) -set(b))

def re_sub(pattern, repl,text):
    return re.sub(pattern, repl, text)



def preprocess_sent(sent):
    sent = re.sub(r"http\S+", "", sent)
    sent = re.sub(r"@\S+", "@user", sent)

    #print(sent)
    sent = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "",sent)
    sent = re_sub(r"/", "",sent)
    sent = emoji.demojize(sent)
    sent = re_sub(r"[:\*]", " ",sent)
    sent = re.sub(' +', ' ', sent)
    sent = re.sub('-', ' ', sent)
    sent = sent.split()
    return ' '.join(sent[:200])


# Given a C/C++ code removes all comments from it
def removeComments(content):
    content = re.sub("-|\d+.", "", content)
    index = 0
    comment_line_inside = False    # this is true when currently index is at a line comment (//)
    comment_block_level = 0     # this is non-zero when currently index is in a block comment (/* */)
    result = []   # this contains the code without the comments
    while index < len(content):
        # on detecting /* change comment_block_level
        if content[index] == '/' and index + 1 < len(content) and content[index+1] == '*':
            comment_block_level += 1
        # on detecting */ change comment block level
        elif content[index] == '/' and content[index-1] == '*':
            comment_block_level -= 1
        # on detecting // set comment_line_inside to True
        elif content[index] == '/' and index + 1 < len(content) and content[index + 1] == '/':
            comment_line_inside = True
        # on detecting \n the line comment is over, so set comment_line_inside to False
        elif content[index] == '\n' and comment_line_inside == True:
            comment_line_inside = False
        # add letters to result if currently no comment is ongoing
        elif not comment_line_inside and comment_block_level == 0:
            result.append(content[index])
        index += 1
    string = ''.join(result)
    string = re.sub('\t|\n', ' ', string)
    string = re.sub(' +', ' ', string)
    return string

In [9]:
def getFreq(lst):
    temp ={}
    for i in lst:
        if i in temp:
            temp[i]+=1
        else:
            temp[i]=1
    return temp

In [10]:
getFreq(list(data['Class']))

{'Not Useful': 3710, 'Useful': 4337}

In [17]:
data = pd.read_csv("IRSE_FIRE_2022_Track_Training_Data_preprocessed.csv")

In [18]:
postId = range(0,len(data))
data['ids'] = postId

In [20]:
data.to_csv("train_val_soft.csv", index = False)

In [24]:
data

,Comments,Surrounding Code Context,Class,ids
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful,0
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful,1
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful,2
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful,3
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful,4
...,...,...,...,...
8042,/*we need an additional DG0-node for a dim-dim...,-10. riaf.back() = true;\n-9. return riaf;...,Useful,8042
8043,/*vertex/line/face domination\n(if fe_other is...,-1. return FiniteElementDomination::no_r...,Useful,8043
8044,/*there are no requirements between continuous...,-2. if (codim > 0)\n-1. if (dynamic_cast...,Useful,8044
8045,/*vertex/line/face domination\n(if fe_other is...,-1. return FiniteElementDomination::no_r...,Useful,8045


In [21]:
testData = pd.read_csv("IRSE_FIRE_2022_Track_Test_Data_1000.csv")

In [26]:
postId = range(len(data), len(data)+len(testData))
testData['ids'] = postId

In [29]:
res = pd.concat([data, testData], ignore_index=True)

In [30]:
res

,Comments,Surrounding Code Context,Class,ids
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful,0
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful,1
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful,2
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful,3
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful,4
...,...,...,...,...
9043,/*The following document where the background ...,-5. #define PNG_CMAP_NONE 0\n\n /*The fol...,Useful,9043
9044,/*Do all the *safe* initialization - 'safe' me...,-1. } png_image_read_control;\n/* Do all the *...,Useful,9044
9045,/*And set the rest of the structure to NULL to...,-10. * called from here must *not* call png_m...,Useful,9045
9046,"/*Use png_ptr here, not info_ptr, because by e...",-8. static png_uint_32\n-7. png_image_format(p...,Useful,9046


In [28]:
testData.to_csv("test_soft.csv", index = False)

In [1]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
#model.to(device)

In [40]:
codeMap ={}
from tqdm import tqdm
for index, row in tqdm(res.iterrows(), total = res.shape[0]):
    if row['ids'] in codeMap:
        continue
    procCode  = removeComments(row['Surrounding Code Context'])
    code_tokens=tokenizer.tokenize(procCode)
    tokens=[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    tokens_ids=tokenizer.convert_tokens_to_ids(tokens)[:300]
    context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]
    codeMap[row['ids']] = context_embeddings[0][0].detach().numpy()

100%|██████████| 9048/9048 [15:48<00:00,  9.54it/s]


In [42]:
import pickle
with open("code_bert_embedding.p", 'wb') as fp:
    pickle.dump(codeMap,fp)

In [43]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('./BERTBASED/')
model = BertModel.from_pretrained("./BERTBASED/")

Some weights of the model checkpoint at ./BERTBASED/ were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [62]:
mBERTEmbedding ={}

from tqdm import tqdm
for index, row in tqdm(res.iterrows(), total = res.shape[0]):
    if row['ids'] in mBERTEmbedding:
        continue
    new_sentence = preprocess_sent(row['Comments'])
    encoded_input = tokenizer(new_sentence, return_tensors='pt')
    output = model(**encoded_input).last_hidden_state
    mBERTEmbedding[row['ids']] = output[0][0].detach().numpy()
    del(output)

100%|██████████| 9048/9048 [03:10<00:00, 47.46it/s]   


In [64]:
import pickle
with open("text_bert_embedding.p", 'wb') as fp:
    pickle.dump(mBERTEmbedding,fp)